In [1]:
!pip install -q efficientnet --quiet

In [2]:
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import gc
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pickle
from tensorflow.keras.callbacks import CSVLogger
import time
from efficientnet.tfkeras import center_crop_and_resize
from tensorflow_addons.losses import TripletSemiHardLoss

In [3]:
from model_semantic import MODEL_INPUT, linear_warmup, build_model_extractor, timecallback, Score_call
from utility_effnet import predictDataset, DataGenerator_mining

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [4]:
dataset_loader_path = '../input/dataset-loader-four/'

Import used dataset

In [5]:
with open(dataset_loader_path + 'train_.pkl', 'rb') as f:
    train_ = pickle.load(f)

In [6]:
MODEL_TYPE = 'B5'
MODEL_ = 'efficientnet'

IMAGE_SIZE = MODEL_INPUT[MODEL_TYPE]
BATCH_SIZE = 128
EPOCHS = 2
NUMBER_OF_IMAGE = 4
EFF_NET_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

In [7]:
training_generator = DataGenerator_mining(train_, BATCH_SIZE, IMAGE_SIZE, number_of_image = NUMBER_OF_IMAGE)

In [8]:
epoch_step = len(training_generator)

total_step = EPOCHS * epoch_step

initial_lr = 0.001
num_warmup_steps = (epoch_step * EPOCHS) //10

optimizer_warmup = linear_warmup(init_lr = initial_lr, num_train_steps = total_step, num_warmup_steps = num_warmup_steps)

In [9]:
model = build_model_extractor(
    optimizer_warmup, EFF_NET_SHAPE, TripletSemiHardLoss(),
    model = MODEL_, modeltype = 'B5', weights = 'noisy-student', trainable = False)

115261440/115255328 [==============================] - 5s 0us/step


In [10]:
history = model.fit(
    training_generator,
    batch_size=BATCH_SIZE, epochs=EPOCHS,
    verbose = 1
)

Epoch 1/2
2260/2260 [==============================] - 11622s 5s/step - loss: 0.6339
Epoch 2/2
2260/2260 [==============================] - 11625s 5s/step - loss: 0.5244


In [11]:
model.save_weights(f'{MODEL_}{MODEL_TYPE}_embedding_model.hdf5')

In [12]:
!mkdir -p saved_model
model.save('saved_model/my_model')